In [15]:
import requests
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

In [6]:
FASTAPI_URL = "http://129.114.27.200:8000/recommend"
payload = {"user_id": "22ml-CTcoabnc-uu4WXXOw"}
num_requests = 100

for _ in range(num_requests):
    response = requests.post(FASTAPI_URL, json=payload)

In [13]:
FASTAPI_URL = "http://129.114.27.200:8000/recommend"
payload = {"user_id": "22ml-CTcoabnc-uu4WXXOw"}

def send_continuous_requests(duration_sec):
    session = requests.Session()
    # mount adapter to support enough connections in the pool
    adapter = requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100)
    session.mount("http://", adapter)

    start = time.time()
    while time.time() - start < duration_sec:
        try:
            session.post(FASTAPI_URL, json=payload, timeout=5)
        except Exception:
            pass  # swallow errors for load test

def run_load_stage(concurrent_workers, duration_sec):
    with ThreadPoolExecutor(max_workers=concurrent_workers) as ex:
        futures = [ex.submit(send_continuous_requests, duration_sec)
                   for _ in range(concurrent_workers)]
        for f in futures:
            f.result()

load_pattern = [1, 2, 3, 5, 3, 2, 1]
delay = 30
for c in load_pattern:
    run_load_stage(c, delay)

In [18]:
FASTAPI_URL = "http://129.114.27.200:8000/recommend"
load_pattern = [1, 3, 5, 10, 5, 3, 1]  # concurrency levels
delay_between_steps = 30  # seconds per stage

def send_continuous_requests(duration_sec):
    session = requests.Session()
    adapter = requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100)
    session.mount("http://", adapter)

    start = time.time()
    while time.time() - start < duration_sec:
        payload = random.choice([{}, {"user_id": "bad data"}])
        try:
            session.post(FASTAPI_URL, json=payload, timeout=5)
        except Exception:
            pass  # ignore errors during load test

def run_load_stage(concurrent_workers, duration_sec):
    with ThreadPoolExecutor(max_workers=concurrent_workers) as executor:
        futures = [
            executor.submit(send_continuous_requests, duration_sec)
            for _ in range(concurrent_workers)
        ]
        for f in futures:
            f.result()

for load in load_pattern:
    print(f"Starting stage: {load} workers for {delay_between_steps}s")
    run_load_stage(load, delay_between_steps)
    print(f"Completed stage: {load} workers\n")

Starting stage: 1 workers for 30s
Completed stage: 1 workers

Starting stage: 3 workers for 30s
Completed stage: 3 workers

Starting stage: 5 workers for 30s
Completed stage: 5 workers

Starting stage: 10 workers for 30s
Completed stage: 10 workers

Starting stage: 5 workers for 30s
Completed stage: 5 workers

Starting stage: 3 workers for 30s
Completed stage: 3 workers

Starting stage: 1 workers for 30s
Completed stage: 1 workers

